In [2]:
import numpy as np
import pandas as pd
import argparse
import sys
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
sys.path.append('/media/dylan/Elements/') #change to path containing nbc repository
from nbc.nbc import NBC

# Load dataset
Requires nbc repository: https://github.com/dylanebert/nbc

In [ ]:
parser = argparse.ArgumentParser()
NBC.add_args(parser)
args = parser.parse_args([
    '--subsample', '1',
    '--dynamic_only', 'True',
    '--train_sequencing', 'token_aligned',
    '--test_sequencing', 'chunked',
    '--features',
    'dist_to_head:most_moving',
    'avg_vel:most_moving',
    'var_vel:most_moving',
    'traj:most_moving',
    'avg_rel:most_moving'
])
nbc = NBC(args)
vgg_embeddings = nbc.get_vgg_embeddings()

**Preprocess vgg embeddings**

In [ ]:
vgg = {'train': {}, 'test': {}}
for type in ['train', 'test']:
    for k, z in vgg_embeddings[type].items():
        z = z[np.arange(0, z.shape[0], 9)]
        start = z[0]; end = z[-1]
        mean = np.mean(z, axis=0); var = np.var(z, axis=0)
        peak = np.max(z, axis=0); trough = np.min(z, axis=0)
        vec = np.concatenate([start, end, mean, var, peak, trough])
        vgg[type][k] = vec

# Nearest neighbor prediction experiment

**Compute predicted token for each test instance**

In [ ]:
def nearest_neighbors(model, features, supervised):    
    train_x = np.vstack(list(features['train'].values()))
    test_x = np.vstack(list(features['test'].values()))
    
    train_tokens = np.array([key[2] for key in features['train'].keys()])
    vocab = {v: k for k, v in enumerate(np.unique(train_tokens))}
    train_y = np.array([vocab[v] for v in train_tokens])
    
    if supervised:
        reducer = LinearDiscriminantAnalysis(n_components=10)
        reducer.fit(train_x, train_y)
    else:
        reducer = TruncatedSVD(n_components=10)
        reducer.fit(train_x)
        
    train_x_reduced = reducer.transform(train_x)
    test_x_reduced = reducer.transform(test_x)
    
    token_means = {k: [] for k in vocab.keys()}
    for i, key in enumerate(features['train'].keys()):
        token = key[2]
        token_means[token].append(train_x_reduced[i])
    for k in token_means.keys():
        token_means[k] = np.mean(token_means[k], axis=0)
    token_means_mat = np.vstack(list(token_means.values()))
    
    results = []
    for key, feat in zip(features['test'].keys(), test_x_reduced):
        dist = cosine_similarity(feat.reshape((1, -1)), np.vstack(token_means_mat))[0]
        indices = np.argsort(dist)[::-1]
        tokens = np.array(list(token_means.keys()))[indices]
        dists = dist[indices]
        res = {
            'model': model,
            'clip': '{} {}-{}'.format(key[0], key[1], key[1] + 450),
            'predicted': tokens[0]
        }
        for i, (token, p) in enumerate(zip(tokens, dists)):
            res['r{}'.format(i+1)] = '{}={:.4f}'.format(token, p)
        results.append(res)
    return pd.DataFrame(results)

def random_neighbors():
    tokens = np.unique(np.array([key[2] for key in features['train'].keys()]))    
    results = []
    for key in nbc.features['test'].keys():
        indices = np.arange(0, len(tokens))
        np.random.shuffle(indices)
        tokens = tokens[indices]
        res = {
            'model': 'rand',
            'clip': '{} {}-{}'.format(key[0], key[1], key[1] + 450),
            'predicted': tokens[0]
        }
        results.append(res)
    return pd.DataFrame(results)

**Run on spatial and cnn models**

In [ ]:
results = []
for model, features, supervised in zip(['spatial_sup', 'spatial_unsup', 'cnn_sup', 'cnn_unsup'],
                                      [nbc.features, nbc.features, vgg, vgg],
                                      [True, False, True, False]):
    results.append(nearest_neighbors(model, features, supervised))
results.append(random_neighbors())

**Add in spatial with names**

In [ ]:
del nbc #may need to free memory

parser = argparse.ArgumentParser()
NBC.add_args(parser)
args = parser.parse_args([
    '--subsample', '1',
    '--dynamic_only', 'True',
    '--train_sequencing', 'token_aligned',
    '--test_sequencing', 'chunked',
    '--features',
    'one_hot_label:most_moving', #add in one-hot encoding of label of most moving object
    'dist_to_head:most_moving',
    'avg_vel:most_moving',
    'var_vel:most_moving',
    'traj:most_moving',
    'avg_rel:most_moving'
])
nbc_with_names = NBC(args)

In [ ]:
results.append(nearest_neighbors('spatial_with_names_sup', nbc_with_names.features, True))
results.append(nearest_neighbors('spatial_with_names_unsup', nbc_with_names.features, False))

**Combine results into single csv for annotation**

In [ ]:
results = pd.concat(results).sample(frac=1)
results = results.sort_values(by='clip')
results.to_csv('neighbors.csv')

# Analyze annotations
After externally annotating predictions and saving to annotations.csv

In [11]:
annotations = pd.read_csv('annotations.csv', index_col=0)[['model', 'clip', 'predicted', 'dylan_label', 'ellie_label']]
mapping = {'y': 1, '/': 0, 'n': 0}
annotations['dylan_label'] = annotations.apply(lambda row: mapping[row['dylan_label']], axis=1)
annotations['ellie_label'] = annotations.apply(lambda row: mapping[row['ellie_label']], axis=1)
annotations['soft_label'] = annotations.apply(lambda row: row['dylan_label'] or row['ellie_label'], axis=1)
annotations['hard_label'] = annotations.apply(lambda row: row['dylan_label'] and row['ellie_label'], axis=1)

**Model precision**

In [15]:
annotations.groupby('model').mean()[['soft_label', 'hard_label']]

,soft_label,hard_label
model,,
cnn_sup,0.349206,0.248677
cnn_unsup,0.396825,0.291005
rand,0.317460,0.227513
spatial_sup,0.396825,0.280423
spatial_unsup,0.195767,0.132275


**Object precision**

In [19]:
annotations = annotations[annotations['model'].isin(['cnn_unsup', 'spatial_unsup', 'spatial_sup'])]

In [21]:
pd.set_option('display.max_rows', 100)
annotations.groupby(['predicted', 'model']).mean()

dylan_label  ellie_label  soft_label  hard_label
predicted model                                                          
DROP      cnn_unsup         0.250000     0.500000    0.500000    0.250000
          spatial_sup       0.000000     0.000000    0.000000    0.000000
          spatial_unsup     0.187500     0.062500    0.187500    0.062500
EAT       cnn_unsup         0.000000     0.000000    0.000000    0.000000
          spatial_sup       0.000000     0.000000    0.000000    0.000000
          spatial_unsup     0.000000     0.000000    0.000000    0.000000
GET       cnn_unsup         0.625000     0.687500    0.750000    0.562500
          spatial_sup       0.615385     0.692308    0.769231    0.538462
          spatial_unsup     0.400000     0.200000    0.600000    0.000000
GIVE      cnn_unsup         0.000000     0.000000    0.000000    0.000000
          spatial_sup       0.000000     0.000000    0.000000    0.000000
          spatial_unsup     0.000000     0.000000    0.000000    0.000000
GO        cnn_unsup         0.275862     0.310345    0.379310    0.206897
          spatial_sup       0.529412     0.529412    0.588235    0.470588
          spatial_unsup     0.000000     0.000000    0.000000    0.000000
HOLD      cnn_unsup         0.636364     0.727273    0.727273    0.636364
          spatial_sup       0.705882     0.764706    0.823529    0.647059
          spatial_unsup     0.800000     1.000000    1.000000    0.800000
OPEN      cnn_unsup         0.000000     0.000000    0.000000    0.000000
          spatial_sup       0.000000     0.000000    0.000000    0.000000
          spatial_unsup     0.000000     0.000000    0.000000    0.000000
PICK      spatial_sup       1.000000     1.000000    1.000000    1.000000
          spatial_unsup     1.000000     1.000000    1.000000    1.000000
PLAY      cnn_unsup         0.285714     0.285714    0.285714    0.285714
          spatial_sup       0.166667     0.333333    0.333333    0.166667
          spatial_unsup     0.176471     0.176471    0.176471    0.176471
PUT       cnn_unsup         0.500000     0.000000    0.500000    0.000000
          spatial_sup       0.354839     0.451613    0.516129    0.290323
          spatial_unsup     0.363636     0.363636    0.454545    0.272727
TAKE      spatial_sup       0.750000     0.833333    0.916667    0.666667
          spatial_unsup     1.000000     1.000000    1.000000    1.000000
THROW     cnn_unsup         0.083333     0.083333    0.083333    0.083333
          spatial_sup       0.062500     0.062500    0.062500    0.062500
          spatial_unsup     0.000000     0.000000    0.000000    0.000000
WALK      cnn_unsup         0.500000     0.625000    0.687500    0.437500
          spatial_sup       0.230769     0.192308    0.269231    0.153846
          spatial_unsup     0.363636     0.363636    0.424242    0.303030
WASH      cnn_unsup         0.478261     0.521739    0.521739    0.478261
          spatial_sup       0.000000     0.000000    0.000000    0.000000
          spatial_unsup     0.000000     0.000000    0.000000    0.000000

**Object counts**

In [27]:
annotations.groupby(['predicted', 'model']).count()[['clip']]

clip
predicted model              
DROP      cnn_unsup         4
          spatial_sup       2
          spatial_unsup    16
EAT       cnn_unsup         2
          spatial_sup      19
          spatial_unsup     2
GET       cnn_unsup        32
          spatial_sup      13
          spatial_unsup     5
GIVE      cnn_unsup        17
          spatial_sup       5
          spatial_unsup    35
GO        cnn_unsup        29
          spatial_sup      17
          spatial_unsup     1
HOLD      cnn_unsup        11
          spatial_sup      17
          spatial_unsup     5
OPEN      cnn_unsup         8
          spatial_sup      10
          spatial_unsup    30
PICK      spatial_sup       4
          spatial_unsup     1
PLAY      cnn_unsup         7
          spatial_sup       6
          spatial_unsup    17
PUT       cnn_unsup         4
          spatial_sup      31
          spatial_unsup    11
TAKE      spatial_sup      12
          spatial_unsup     3
THROW     cnn_unsup        36
          spatial_sup      16
          spatial_unsup    25
WALK      cnn_unsup        16
          spatial_sup      26
          spatial_unsup    33
WASH      cnn_unsup        23
          spatial_sup      11
          spatial_unsup     5